# Australian Weather App Project


## - Purpose

To pull a meteorological data from a meteorological dataset, and have it displayed for public view.
Due to inability to obtain the latest dataset, we obtained a dataset of a month in 2017, and acted as if it is the latest data.


## - What contains in the app

- Simple weather request and reply
- Detailed weather request and reply
-- with the option of detailed reply of a day, or overall view of the month
- 'Holiday Planner'


## - Reference:

- cloud = https://en.wikipedia.org/wiki/Okta#:~:text=In%20meteorology%2C%20an%20okta%20is,8%20oktas%20(completely%20overcast).
- https://stackoverflow.com/questions/11707586/how-do-i-expand-the-output-display-to-see-more-columns-of-a-pandas-dataframe
- https://stackoverflow.com/questions/30576323/pandas-convert-index-values-to-lowercase
- https://www.w3schools.com/python/gloss_python_while_else.asp
- https://www.geeksforgeeks.org/string-capitalize-python/

In [2]:
# import the necessary packages

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime 

pd.set_option('display.width', 1000) # to expand the width of the table

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

In [3]:
# pull data

weather = pd.read_csv('../data/weatherAUS255265.csv')
weather.sort_values(by = ['Location', 'Date'], inplace = True)
weather.set_index('Location', inplace = True)

weather.head(10)

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
Location,,,,,,,,,,,,,,,,,,,,,
Adelaide,5/25/2017,10.0,20.2,0.2,NaN,NaN,WSW,26.0,NE,W,...,79.0,53.0,1023.2,1020.9,NaN,NaN,14.2,19.4,No,No
Adelaide,5/26/2017,12.5,21.9,0.0,NaN,NaN,NW,24.0,E,NW,...,87.0,36.0,1023.0,1018.1,NaN,NaN,13.9,21.6,No,No
Adelaide,5/27/2017,11.6,20.4,0.0,NaN,NaN,WSW,48.0,NNE,NW,...,61.0,52.0,1015.1,1011.0,NaN,NaN,15.0,19.5,No,Yes
Adelaide,5/28/2017,9.5,15.1,16.6,NaN,NaN,WSW,50.0,SW,WSW,...,89.0,58.0,1015.5,1015.8,NaN,NaN,11.5,14.9,Yes,Yes
Adelaide,5/29/2017,7.9,15.2,3.8,NaN,NaN,W,28.0,N,WNW,...,95.0,93.0,1023.1,1021.6,NaN,NaN,10.8,14.1,Yes,Yes
Adelaide,5/30/2017,9.7,15.1,16.0,NaN,NaN,S,35.0,S,SSE,...,71.0,40.0,1028.9,1029.3,NaN,NaN,12.2,14.0,Yes,No
Adelaide,5/31/2017,4.8,15.0,0.0,NaN,NaN,E,24.0,NaN,E,...,68.0,40.0,1036.6,1035.2,NaN,NaN,9.4,14.4,No,No
Adelaide,6/1/2017,2.9,15.4,0.0,NaN,NaN,ENE,17.0,ENE,SW,...,58.0,48.0,1037.7,1035.5,NaN,NaN,10.1,14.5,No,No
Adelaide,6/10/2017,8.2,16.2,0.0,NaN,NaN,WSW,24.0,NaN,WSW,...,69.0,55.0,1035.6,1032.5,NaN,NaN,12.2,15.1,No,No


In [3]:
# tuple for cities for verifying locations

auscities = tuple(weather.index.unique())

# dictionary of admin user/pass
userpass = {'admin' : 'admin',
            'test' : 'test',
            'success' : 'fail'}


In [342]:
# verifying if the number of cities input was correct

if len(auscities) == len(list(weather.index.unique())):
    print(True)
    print(f"{len(auscities)}")
else:
    print(False)


True
49


In [343]:
class Weather(): # Class-making
    
    def __init__(self, location): 
        self.location = location
    
    
    def __temptable(self, templocation): # make a temp table (since we will only 1 location at a time, a temp table is possible)
        self.templocation = templocation
        self.temp = weather.loc[[self.templocation]] 
        self.temp2 = self.temp.copy(deep = True) # in case we need to add or modify the dbase
        self.temp2.set_index('Date', inplace = True)
        self.temp2.sort_index()
        self.__tempweather(self.templocation)
        self.new_date = []
        self.new_date_str = []
        
        for date in self.temp2.index: # new date
            self.new_date.append(datetime.strptime(date, '%m/%d/%Y'))
        self.temp2['new_date'] = self.new_date
        
        for date2 in self.temp2['new_date']: # new date str
            self.new_date_str.append(datetime.strftime(date2, '%m/%d/%Y'))
        self.temp2['new_date_str'] = self.new_date_str
        self.temp2.sort_values(by = 'new_date_str')

        
    def __tempweather(self, templocation):
        self.templocation = templocation
        
        self.weatherlist9am = []
        self.weatherlist3pm = []
        
        for date in self.temp2.index: # make a 'Weather9am' and 'Weather3pm' column, to make some functions easier #
        
            if self.temp2.at[date, 'RainToday'] == "Yes" or self.temp2.at[date, 'Rainfall'] >= 1: # rainy
                self.weatherlist9am.append("Rainfall")
            
            elif self.temp2.at[date,'Cloud9am'] < 1: # sunny
                self.weatherlist9am.append("Clear")
            
            elif self.temp2.at[date, 'Cloud9am'] <= 2: # mostly sunny
                self.weatherlist9am.append("Mostly Clear")
            
            elif self.temp2.at[date, 'Cloud9am'] <= 5: # partly cloudy
                self.weatherlist9am.append("Partly Cloudy")
            
            elif self.temp2.at[date, 'Cloud9am'] <= 7: # mostly cloudy
                self.weatherlist9am.append("Mostly Cloudy")
           
            elif self.temp2.at[date, 'Cloud9am'] == 8: # overcast
                self.weatherlist9am.append("Overcast")
            
            elif self.temp2.at[date, 'Cloud9am'] != "Yes" and self.temp2.at[date, 'Cloud9am'] != "No" and self.temp2.at[date, 'Rainfall'] < 1: # Non-Yes/No
                self.weatherlist9am.append("Clear") 
            
            else:
                self.weatherlist9am.append("Not Enough Data") # for cases which fall out of all conditions
                    
        
            if self.temp2.at[date, 'RainToday'] == "Yes" or self.temp2.at[date, 'Rainfall'] >= 1: # rainy
                self.weatherlist3pm.append("Rainfall")
           
            elif self.temp2.at[date, 'Cloud3pm'] < 1: # sunny
                self.weatherlist3pm.append("Clear")
            
            elif self.temp2.at[date, 'Cloud3pm'] <= 2: # mostly sunny
                self.weatherlist3pm.append("Mostly Clear")
            
            elif self.temp2.at[date, 'Cloud3pm'] <= 5: # partly cloudy
                self.weatherlist3pm.append("Partly Cloudy")
            
            elif self.temp2.at[date, 'Cloud3pm'] <= 7: # mostly cloudy
                self.weatherlist3pm.append("Mostly Cloudy")
            
            elif self.temp2.at[date, 'Cloud3pm'] == 8: # overcast
                self.weatherlist3pm.append("Overcast")
            
            elif self.temp2.at[date, 'Cloud3pm'] != "Yes" and self.temp2.at[date, 'Cloud3pm'] != "No" and self.temp2.at[date, 'Rainfall'] < 1: # Non-Yes/No
                self.weatherlist3pm.append("Clear") 
            
            else:
                self.weatherlist3pm.append("Not Enough Data") # for cases which fall out of all conditions
            
        self.temp2['Weather9am'] = self.weatherlist9am   
        self.temp2['Weather3pm'] = self.weatherlist3pm
            
        
    def simple_weather(self, location = 'null'): # weathers for date and location
        self.location = location
        counter = 0
        
        if self.location in auscities: 
            self.__temptable(self.location) 
            userinputtime = 'filler'
            
            while userinputtime: # for loop start
                userinputtime = input("Please insert the date (mm/dd/yyyy): ") # date input
                
                if userinputtime: # if true
                    if userinputtime in self.temp2.index.tolist(): # if the date is correct
                        print(f"\nThe weather for {userinputtime} at {self.location} is {self.temp2.at[userinputtime, 'Weather9am']} in the morning, and {self.temp2.at[userinputtime, 'Weather3pm']} in the afternoon.") ## need to make the 
                        break
                        
                    elif userinputtime not in self.temp2.index.tolist(): # if not correct or data not available
                        counter += 1
                        print("Invalid input.")
                        if counter == 3:
                            print("Maximum attempt reached.")
                            break
                
                else: # if false
                    print("No input detected. The app will now shutdown.")
                    break
            else:
                print("No input detected. The app will now shutdown.")
                
                
                
        elif self.location == 'null' or self.location != True:
            counter += 1
            print("The input is not valid.")
            
            while self.location == 'null' or self.location not in auscities:
                userinput = input("Please enter a valid city within Australia (Eg: GoldCoast, Launceston): ")
                
                if userinput in auscities:
                    self.simple_weather(userinput)
            
                else:
                    counter += 1
                    print("Invalid input.")
                    if counter == 3:
                        print("Maximum attempt reached.")
                        break
                
        else:
            print("The input is not a valid city within Australia.")
            
        

        
    def detailed_weather(self, location = 'null'):
        self.location = location
        counter = 0
        convertdate = ''
        
        if self.location in auscities: 
            self.__temptable(self.location) 
            userinputtime = 'filler'
            
            while userinputtime: # for loop start
                userinputtime = input("Please insert the date (mm/dd/yyyy or 'overall'): ") # date input  
                
                try: # a try-except filter, to separate int
                    int_test = int(userinputtime)
                    print("Invalid input. Integer, not Date.")
                    counter += 1
                    
                    if counter == 3:
                        print("Maximum attempt reached.")
                        break

                except ValueError:
                
                    if userinputtime == str: #in case it is a str, then lowercase it, and ignore if not
                        userinputtime = userinputtime.lower()
                    
                    if userinputtime == 'overall': # if overall (a month)
                                self.__overall()
                                break
                    
                    try: # a try-except filter, to separate date
                        convertdate = datetime.strptime(userinputtime, '%m/%d/%Y')
                        
                        if userinputtime:

                            if userinputtime in self.temp2.index.tolist(): # if date
                                print(f"\nThe weather for {userinputtime} at {self.location} is {self.temp2.at[userinputtime, 'Weather9am']} at {self.temp2.at[userinputtime, 'Temp9am']}°C and {self.temp2.at[userinputtime, 'Humidity9am']}% humidity in the morning,"
                                      f"\nand {self.temp2.at[userinputtime, 'Weather3pm']} at {self.temp2.at[userinputtime, 'Temp3pm']}°C and {self.temp2.at[userinputtime, 'Humidity3pm']}% humidity in the afternoon."
                                      f"\nDo note that the temperature can go as high as {self.temp2.at[userinputtime, 'MaxTemp']}°C")
                                break

                            else:
                                counter +=1
                                
                                if counter == 3:
                                    print("Maximum attempt reached.")
                                    break

                                else:
                                    print("Invalid input. Date outside database range. Current range = 5/25/2017 - 6/24/2017.")

                    except ValueError:
                        if userinputtime:
                            print("Invalid input. No such function.")
                            counter += 1
                        
                        else:
                            print("No input detected. The app will now shutdown.")
                            break
                        
                        if counter == 3:
                            print("Maximum attempt reached.")
                            break
                       

            else:
                print("No input detected. The app will now shutdown.")

        elif self.location == 'null' or self.location != True:
            counter += 1
            print("The input is not valid.")

            while self.location == 'null' or self.location not in auscities:
                userinput = input("Please enter a valid city within Australia (Eg: GoldCoast, Launceston): ")

                if userinput in auscities:
                    self.detailed_weather(userinput)

                else:
                    counter += 1
                    print("Invalid input.")
                    if counter == 3:
                        print("Maximum attempt reached.")
                        break

        else:
            print("The input is not a valid city within Australia.")

        

    
    def holiday_planner(self, location = 'null'): ## 3 - Trip planner
        self.location = location
        counter = 0
        counter1 = 0
        counter2 = 0
        self.sunnyday = []
        done_tag = False
        
        if self.location in auscities: 
            self.__temptable(self.location) 
            self.uinputdate1 = 'filler'
            self.uinputdate2 = 'filler'
            
            while self.uinputdate1:
                self.uinputdate1 = input("Please enter the earliest date for the holiday (mm/dd/yyyy): ")
                  
                if self.uinputdate1:
                    try: # a try-except filter, to separate int from date
                        int_test = int(self.uinputdate1)
                        print("Invalid input. Integer, not Date.")
                        counter += 1
                        if counter == 3:
                            print("Maximum attempt reached.")
                            break

                    except ValueError:
                        
                        if self.uinputdate1 in self.temp2['new_date']: ##
                            self.newuinputdate1 = datetime.strptime(self.uinputdate1, '%m/%d/%Y')
                            
                            while self.uinputdate2:
                                self.uinputdate2 = input("Please enter the latest date for the holiday (mm/dd/yyyy): ")

                                if self.uinputdate2:
                                    try: # another try-except filter
                                        int_test = int(self.uinputdate2)
                                        print("Invalid input. Integer, not Date.")
                                        counter += 1
                                        if counter == 3:
                                            print("Maximum attempt reached.")
                                            break

                                    except ValueError or AttributeError:

                                        if self.uinputdate2 in self.temp2.index: #self.temp2['new_date']: ##
                                            self.newuinputdate2 = datetime.strptime(self.uinputdate2, '%m/%d/%Y')
                                            
                                            for date in self.temp2.index:

                                                if self.newuinputdate1 <= self.temp2.at[date, 'new_date'] and self.newuinputdate2 >= self.temp2.at[date, 'new_date']:

                                                    if self.temp2.at[date, 'Weather9am'] == "Clear" or self.temp2.at[date, 'Weather9am'] == "Mostly Clear": # filter weather
                                                        self.sunnyday.append(self.temp2.at[date, 'new_date_str'])

                                            print(f"The dates that the weather are either Clear or Mostly Clear, and are suitable for your holidays at {self.location} are:")

                                            self.sunnyday.sort()

                                            for i in range(len(self.sunnyday)): # -- shows days that are mostly sunny
                                                print(self.sunnyday[i])
  
                                            done_tag = True

                                            print("Wishing you a super holiday!")
                                            break

                                        elif counter2 == 3:
                                            print("Maximum attempt reached.")
                                            break

                                        else:
                                            counter2 +=1
                                            print("Invalid input. Date outside database range. Current range = 5/25/2017 - 6/24/2017.")

                            else: # if false while1
                                    print("No input detected. The app will now shutdown.")
                                    break
                                    
                            if done_tag:
                                break

                        elif counter1 == 3:
                            print("Maximum attempt reached.")
                            break

                        else:
                            counter1 +=1
                            print("Invalid input. Date outside database range. Current range = 5/25/2017 - 6/24/2017.")  


            else: # if false for while2
                print("No input detected. The app will now shutdown.")
                
            
        elif self.location == 'null' or self.location != True:
            print("The input is not valid (Invalid Location).")
            
            while self.location == 'null' or self.location not in auscities:
                userinput = input("Please enter a valid city within Australia (Eg: GoldCoast, Launceston): ")
                
                if userinput in auscities:
                    self.holiday_planner(userinput)
            
                else:
                    counter += 1
                    print("Invalid input.")
                    if counter == 3:
                        print("Maximum attempt reached.")
                        break
            
        else:
            print("The input is not a valid city within Australia.")
                          

    
    def __overall(self):
        self.temp2['MaxTemp'].interpolate(inplace = True) # linear interpolation to deal with null values
        self.temp2['MinTemp'].interpolate(inplace = True)
        self.temp2['Rainfall'].interpolate(inplace = True)
        self.temp2['Temp9am'].interpolate(inplace = True)
        self.temp2['Temp3pm'].interpolate(inplace = True)
        
        sorteddbase = self.temp2.sort_values(by = 'new_date_str', inplace = True)
        ysection1a = self.temp2['MaxTemp']
        ysection1b = self.temp2['MinTemp']
        ysection2 = self.temp2['Rainfall']
        xsection = self.temp2['new_date_str']

        plt.plot(xsection, ysection1a) # plotting the Max/Min Temp graph
        plt.plot(xsection, ysection1b)
        
        plt.title(f"Min and Max Temperature of {self.location}") # title
        plt.ylabel('Temperature (°C)')
        plt.xlabel('Date')
        plt.xticks(rotation='vertical')
        plt.subplots_adjust(bottom=0.15)
        plt.legend(['Max Temperature', 'Min Temperature'])
        plt.show()
              
        print(f"\n") # just for spacing
        
        plt.plot(xsection, ysection2)

        plt.title(f"Rainfall Precipitation of {self.location}")
        plt.ylabel('Precipitation (mm)')
        plt.xlabel('Date')
        plt.xticks(rotation='vertical')
        plt.subplots_adjust(bottom=0.15)
        #plt.legend(['']) # 1 data, so don't really need a legend.
        plt.show()
        
        displaytable = self.temp2[['MinTemp', 'MaxTemp', 'Rainfall', 'Weather9am', 'Weather3pm']].copy()
        displaytable.rename(columns = {'MinTemp' : 'Min Temperature (°C)', 
                                       'MaxTemp' : 'Max Temperature (°C)', 
                                       'Rainfall' : 'Precipitation (mm)',
                                       'Weather9am' : 'Weather at 9am', 
                                       'Weather3pm' : 'Weather at 3pm'}, inplace = True)
        print(displaytable)
 

In [365]:
# UI

def UI():
    uinput = 'filler'
    uinput2 = 'filler'
    uinput3 = 'filler'
    uinputname = 'filler'
    uinputpass = 'filler'
    counter = 0
    
    
    uinput = input('''Thank you for using the Australian Weather App!
                    \nPlease input the number that corresponds to of the desired function: 
                    \n 1. Simple Weather Report 
                    \n 2. Holiday Planner 
                    \n 3. Detailed Weather Report (Admin Access Required)
                    \n''')

    if uinput == '1':

        while uinput2:
            uinput2 = input("Please enter your city (Eg: GoldCoast, Launceston): ")

            if uinput2:
                simpleweather = Weather(uinput2)
                simpleweather.simple_weather(uinput2)
                break
            

        else:
            print(f"No input detected.")
            

    elif uinput == '2':

        while uinput2:
            uinput2 = input("Please enter the city for your holiday plans (Eg: GoldCoast, Launceston): ")

            if uinput2:
                holidayplan = Weather(uinput2)
                holidayplan.holiday_planner(uinput2)
                break
            

        else:    
            print(f"No input detected.")
            

    elif uinput == '3':
        done_tag = False
        
        while uinputname:
            uinputname = input("Please enter your credentials (username): ")

            if uinputname:

                while uinputpass:
                    uinputpass = input("Please enter your credentials (password): ") 

                    if uinputpass:
                        done_tag = True
                        break
                        
                else:
                    print(f"No input detected.")
                    break
                
                if done_tag == True:
                    break


        else:
            print(f"No input detected.")


        if uinputname != '' and uinputpass != '':
            passcounter = 0
            
            for key, value in userpass.items():

                if uinputname == key and uinputpass == value: # security filter/test
                    print("Credentials validated. Access granted.")
                    
                    while uinput3:
                        uinput3 = input("Please enter the city (Eg: GoldCoast, Launceston): ")
                        
                        if uinput3:
                            detailedweather = Weather(uinput3)
                            detailedweather.detailed_weather(uinput3)
                            break
                   
                    else:    ##
                        print(f"No input detected.")
                        

                else:
                    passcounter +=1

            if passcounter == len(userpass):
                print("Invalid username or password.")

        else:
            print("Access denied.")

    elif uinput:
        counter += 1
        print("Invalid input.")
        
        if counter == 3:
            print("Maximum attempt reached.")
        

    else:
        print("No input detected. The app will shutdown.")

    print(f"\nThank you for using Australian Weather App!")
    

In [368]:
UI()

Thank you for using the Australian Weather App!
                    
Please input the number that corresponds to of the desired function: 
                    
 1. Simple Weather Report 
                    
 2. Holiday Planner 
                    
 3. Detailed Weather Report (Admin Access Required)
                    
1
Please enter your city (Eg: GoldCoast, Launceston): 
No input detected.

Thank you for using Australian Weather App!
